In [5]:
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
warnings.filterwarnings(action='ignore')

random_state = 7

In [6]:
def modify_anomal(row):
    product_name = row.product_name
    if product_name == '[1000개한정미정당 스파이시로제떡볶이 376gx2개':
        product_name = '[1000개한정]미정당 스파이시로제떡볶이 376gx2개'
    return product_name

def find_bracket(row):
    product_name = row.product_name
    list_in = ''

    if product_name.find('[') >= 0:
        start = product_name.find('[')
        if product_name.find(']') > start:
            end = product_name.find(']')
            list_in = product_name[start:end+1]

    return list_in

def find_holiday(weekday):
    if weekday == 5 or weekday == 6:
        weekday="주말"
    else:
        weekday="평일"    
    return weekday

In [7]:
train_data = pd.read_csv('./data/tmk_bda_train.csv', index_col=0)
product_info = pd.read_csv('./product_info.csv')
train_data = train_data.merge(product_info, on='product_name', how='left')

# 상품명 오기 처리
train_data['product_name'] = train_data.apply(modify_anomal, axis=1)

# [UPCYCLE], [2023설사원선물신청], [냉동]과 같이 괄호안에 있는 항목 추출
train_data['list_in'] = train_data.apply(find_bracket, axis=1)

# 'list_in' 컬럼 빈도수 인코딩
encoder = CountEncoder(cols="list_in", normalize=True)
encoder.fit(train_data["list_in"])

CountEncoder(cols=['list_in'], combine_min_nan_groups=True, normalize=True)

In [8]:
ori_data = pd.read_csv('./data/tmk_bda_test.csv', index_col=0)
product_info = pd.read_csv('./product_info.csv')
ori_data = ori_data.merge(product_info, on='product_name', how='left')



# 상품명 오기 처리
ori_data['product_name'] = ori_data.apply(modify_anomal, axis=1)

# [UPCYCLE], [2023설사원선물신청], [냉동]과 같이 괄호안에 있는 항목 추출
ori_data['list_in'] = ori_data.apply(find_bracket, axis=1)

# 성별 + 나이
# ori_data["gen_age"] = ori_data["gender"] + ori_data["age_grp"].astype("str")

# week
ori_data['order_date'] = pd.to_datetime(ori_data['order_date'], format='%Y%m%d')
ori_data['week'] = ori_data['order_date'].dt.isocalendar().week + 1
ori_data.loc[ori_data['week'] == 53, 'week'] = 1 # 22년 53주차 -> 23년 1주차 #### 53주차인지 54주차인지 확인 필요
ori_data["week"] = ori_data["week"].astype("int")

# 평/휴일
ori_data["holiday"] = ori_data["order_date"].apply(lambda x: x.weekday())
ori_data["holiday"] = ori_data["holiday"].apply(find_holiday)

# 같은 주문번호내의 카테고리 별 주문 금액
# ! 이걸 하려면 train, valid의 주문번호가 섞이면 안된다.
# ! 가격도 정확하고 할인율도 정확해서 주문번호 섞이지 않게 해도 될듯?
# ori_data.groupby(['scd', 'category']).agg({'price': 'sum'})
# category_price = 
# ori_data = ori_data.merge(category_price, on='scd', how='left')

# 가격 관련
## 스케일링 전 가격
ori_data['price'] = ori_data.net_order_amt.apply(lambda x: math.e ** x)
## 개당 정가
# todo - 1. 1개만 하는게 아니라 qty로 나눈 값으로 할지
# todo - 2. 정가 가격을 최대 가격으로 할지, 가장 빈도가 많은 특정 가격으로 할지
# qty_1 = ori_data[ori_data.net_order_qty == 1]
# ori_data['qty1_price'] = ori_data['price'] / ori_data['net_order_qty']
# unit_price = ori_data.groupby('product_name')['qty1_price'].max().reset_index().rename(columns={'qty1_price': 'unit_price'})
# ori_data = ori_data.merge(unit_price, on='product_name', how='left')
## 할인율
ori_data['discount_rate'] = 1 - (ori_data['price'] / (ori_data['unit_price'] * ori_data['net_order_qty']))
## 주문번호 별 총 주문 금액
# ori_data['price_sum'] = ori_data.groupby('scd')['price'].transform('sum')

ori_data.drop(['order_date', 'net_order_amt', 'net_order_qty', 'product_name'], axis=1, inplace=True)
ori_data

,scd,gender,age_grp,employee_yn,prime_yn,unit_price,category,list_in,week,holiday,price,discount_rate
0,20230101964282,M,4,N,NaN,6980.500000,국/김치/김/반찬/두부,,1,주말,3505.0,0.497887
1,20230101970142,M,4,Y,NaN,3794.333333,핫도그/떡볶이/간식,,1,주말,3291.0,0.132654
2,20230101965237,F,4,N,NaN,6901.000000,밥/죽/면,,1,주말,3934.0,0.429938
3,20230102973798,F,3,N,NaN,5481.000000,국/김치/김/반찬/두부,,2,평일,9371.0,0.430092
4,20230101965633,F,3,N,NaN,5481.000000,국/김치/김/반찬/두부,,1,주말,5504.0,0.497902
...,...,...,...,...,...,...,...,...,...,...,...,...
19655,20230103986821,M,5,Y,NaN,4050.500000,만두/피자/치킨,,2,평일,2781.0,0.313418
19656,20230102979023,F,3,Y,NaN,5209.000000,양념/소스/가루/오일,,2,평일,4805.0,0.077558
19657,20230103989581,F,3,Y,NaN,1350.500000,핫도그/떡볶이/간식,,2,평일,12348.0,0.085672
19658,20230104999357,M,4,N,NaN,9481.000000,돈까스/함박/구이,,2,평일,5404.0,0.430018


In [9]:
# 빈도수 인코딩
ori_data["list_frequency"] = encoder.transform(ori_data["list_in"])

# 라벨 인코딩 : prime_yn, gen_age, week, weeks_in_month
le = LabelEncoder()
ori_data["category"] = le.fit_transform(ori_data["category"])

ori_data["holiday"] = le.fit_transform(ori_data["holiday"])
# ori_data["gen_age"] = le.fit_transform(ori_data["gen_age"])
ori_data["gender"] = le.fit_transform(ori_data["gender"])
ori_data["age_grp"] = le.fit_transform(ori_data["age_grp"])
ori_data["prime_yn"] = le.fit_transform(ori_data["prime_yn"])

# 임직원, 일반 데이터 분리
emp_data = ori_data[ori_data.employee_yn == 'Y']
cus_data = ori_data[ori_data.employee_yn == 'N']

emp_data.reset_index(drop=True, inplace=True)
cus_data.reset_index(drop=True, inplace=True)

emp_data.drop(columns='employee_yn', inplace=True)
cus_data.drop(columns='employee_yn', inplace=True)

emp_scd = pd.DataFrame(emp_data.scd)
emp_data.drop(['scd', 'list_in', 'prime_yn'], axis=1, inplace=True)
cus_scd = pd.DataFrame(cus_data.scd)
cus_data.drop(['scd', 'list_in', 'prime_yn'], axis=1, inplace=True)

In [10]:
emp_data

,gender,age_grp,unit_price,category,week,holiday,price,discount_rate,list_frequency
0,1,3,3794.333333,11,1,0,3291.0,0.132654,0.648109
1,1,3,11125.000000,4,1,0,8750.0,0.213483,0.017766
2,0,2,21511.000000,6,2,1,20507.0,0.046674,0.648109
3,1,3,11701.000000,9,2,1,23401.0,0.000043,0.648109
4,1,3,9981.000000,1,1,0,17965.0,0.100040,0.648109
...,...,...,...,...,...,...,...,...,...
8041,1,4,9980.500000,4,2,1,6227.0,0.376083,0.648109
8042,1,2,5481.000000,6,2,1,9865.0,0.100073,0.648109
8043,1,4,4050.500000,4,2,1,2781.0,0.313418,0.648109
8044,0,2,5209.000000,9,2,1,4805.0,0.077558,0.648109


In [11]:
emp_xgb = XGBClassifier()
emp_xgb.load_model('./emp_model.model')
emp_preds = emp_xgb.predict(emp_data)
emp_pred_proba = emp_xgb.predict_proba(emp_data)[:, 1]

print(sum(emp_preds == 0), sum(emp_preds == 1))

3168 4878


In [12]:
cus_xgb = XGBClassifier()
cus_xgb.load_model('./cus_model.model')
cus_preds = cus_xgb.predict(cus_data)
cus_pred_proba = cus_xgb.predict_proba(cus_data)[:, 1]

print(sum(cus_preds == 0), sum(cus_preds == 1))

6480 5134


# 제출 csv 생성

In [13]:
emp_scd['prime_yn'] = emp_preds
cus_scd['prime_yn'] = cus_preds

In [14]:
cus_scd

,scd,prime_yn
0,20230101964282,1
1,20230101965237,0
2,20230102973798,0
3,20230101965633,1
4,20230101963753,0
...,...,...
11609,20230102980326,1
11610,20230102983068,0
11611,20230103992835,1
11612,20230104999357,0


In [15]:
submission = pd.read_csv('./data/tmk_bda_test.csv', index_col=0)
submission['prime_yn'] = pd.concat((emp_scd, cus_scd), axis=0)['prime_yn'].values
submission = submission.sort_values(by='scd')
submission

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn
49490,20230101963221,꼬마새우까스 250g,1,7.959975,F,3,N,20230101,0
56951,20230101963226,고메 바삭튀겨낸모짜렐라돈카츠450g,2,9.845064,M,3,Y,20230101,0
50897,20230101963226,햇반컵반 BIG바질크림리조또 306g,2,9.284055,M,3,Y,20230101,1
56921,20230101963226,햇반소프트밀 들깨버섯죽 420g,2,8.995289,M,3,Y,20230101,1
50954,20230101963226,햇반소프트밀 해물누룽지죽 420g,2,9.196748,M,3,Y,20230101,0
...,...,...,...,...,...,...,...,...,...
65148,20230131216338,햇반솥반 전복내장영양밥 200g,1,7.974189,M,4,N,20230131,1
65182,20230131216371,고메 바삭쫄깃한 사천탕수육 450g,1,8.746557,F,3,N,20230131,0
60330,20230131216371,밥이랑야채 27g,1,7.273093,F,3,N,20230131,0
65176,20230131216446,고메 중화짜장 760g,1,8.655214,F,4,N,20230131,1


In [16]:
# 같은 주문번호인 상품들의 프라임 예측의 평균이 0.5이상이면 모두 프라임으로 예측
pred_df = submission.loc[:,['scd', 'prime_yn']].rename(columns={'prime_yn': 'ori_prime_yn'})
preds = pred_df.groupby('scd')['ori_prime_yn'].mean().agg(round).reset_index().rename(columns={'ori_prime_yn': 'prime_yn'})
pred_df = pred_df.merge(preds, on='scd', how='left')
submission['prime_yn'] = pred_df['prime_yn'].values

In [17]:
submission.to_csv('./test_제일세당.csv', index=False)